In [1]:
import os
os.environ["THEANO_FLAGS"]="mode=FAST_RUN, device=cpu,floatX=float32"
import theano
import keras
import time

import numpy as np
import Binary_DUDE as DUDE
import Binary_N_DUDE as N_DUDE

from numpy import *
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, RMSprop, Adagrad, Adam, Adadelta
from keras.utils import np_utils

Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [21]:
data=np.load('./Neural_dude_Data_lena512.npz')
z_two=data['z_two']
z=data['z']
imarray=data['imarray']
n=imarray.shape[0]*imarray.shape[1]
offset=data['offset']
L=data['L']
L_new=data['L_new']

In [8]:
### Make data which this experiment needs ###
n=imarray.shape[0]*imarray.shape[1]
alpha_size=2
mapping_size=3
nb_classes=2

Z=[]
P=np.zeros((imarray.shape[0],imarray.shape[1],nb_classes),dtype=np.int)
Z=np_utils.to_categorical(z[9],nb_classes) ## [0]->[1 0], [1]->[0 1]
P=np.reshape(Z,(imarray.shape[0], imarray.shape[1], nb_classes))

In [76]:
######## 1-D DUDE ########
def One_DUDE(z,k,delta):
    n=len(z)
    x_hat=np.zeros(n,dtype=np.int)
    s_hat=x_hat.copy()
    th_0=2*delta*(1-delta)
    th_1=delta**2+(1-delta)**2
    m={}
    for i in range(k,n-k):
        context=z[i-k:i].tolist()+z[i+1:i+k+1].tolist()
        context_str = ''.join(str(e) for e in context)
        if not m.has_key(context_str):
            m[context_str]=np.zeros(2,dtype=np.int)
            m[context_str][z[i]]=1
        else:
            m[context_str][z[i]]+=1
        ratio = float(m[context_str][1]) / float(np.sum(m[context_str]))
        if ratio < th_0:
            s_hat[i]=1
        elif ratio >= th_1:
            s_hat[i]=2
        else:
            s_hat[i]=0
    return s_hat

In [135]:
z_new=np.reshape(z[9],(imarray.shape[0],imarray.shape[1]))
print z_new

[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 1 0 1]
 [0 0 0 ..., 0 1 1]]


In [136]:
test_s_hat_DUDE=One_DUDE(z[9],1,0.01)
test_s_hat_DUDE=np.reshape(test_s_hat_DUDE,(imarray.shape[0], imarray.shape[1]))
test_s_hat_DUDE4=One_DUDE(z[9],4,0.01)
test_s_hat_DUDE4=np.reshape(test_s_hat_DUDE4,(imarray.shape[0], imarray.shape[1]))
test_s_hat_DUDE11=One_DUDE(z[9],11,0.01)
test_s_hat_DUDE11=np.reshape(test_s_hat_DUDE11,(imarray.shape[0], imarray.shape[1]))


In [137]:
print test_s_hat_DUDE, test_s_hat_DUDE.shape
print test_s_hat_DUDE4, test_s_hat_DUDE4.shape
print test_s_hat_DUDE11, test_s_hat_DUDE11.shape

[[0 2 2 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] (512, 512)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 2 0]
 [0 0 0 ..., 0 2 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] (512, 512)
[[0 0 0 ..., 2 2 2]
 [2 2 2 ..., 2 2 2]
 [2 2 0 ..., 2 2 2]
 ..., 
 [1 0 1 ..., 1 0 1]
 [1 1 0 ..., 2 1 2]
 [1 0 1 ..., 0 0 0]] (512, 512)


In [144]:
######## 2-D DUDE ########
def Two_DUDE(z_two, k, delta, n, offset):
    width=int(math.sqrt(n))
    th_0=2*delta*(1-delta)
    th_1=delta**2+(1-delta)**2
    s_hat=zeros((width,width),dtype=np.int)
    
    if k==1:
        C=zeros((n-2*width,1),dtype=np.int)
        for i in range(0,width):
            for j in range(1,width-1):
                context=[]
                m={}
                for l in range(1,k+1):
                    [a_i, a_j]=[i,j]+offset[2*l-2] #clear
                    [b_i, b_j]=[i,j]+offset[2*l-1] #clear
                    context=context+[z_two[a_i,a_j],z_two[b_i,b_j]]
                context_str = ''.join(str(e) for e in context)
                if not m.has_key(context_str):
                    m[context_str]=np.zeros(2,dtype=np.int)
                    m[context_str][z_two[i,j]]=1
                else:
                    m[context_str][z_two[i,j]]+=1
                ratio = float(m[context_str][1]) / float(np.sum(m[context_str]))
                if ratio < th_0:
                    s_hat[i,j]=1
                elif ratio >= th_1:
                    s_hat[i,j]=2
                else:
                    s_hat[i,j]=0
                C[i*(width-2)+j-1]=context_str
        #s_hat=np.reshape(s_hat,(n,))
        return s_hat, C
    
    
    elif 2<=k<=4:
        C=zeros((n-2*(width)-2*(width-2),1),dtype=np.int)
        for i in range(1,width-1):
            for j in range(1,width-1):
                context=[]
                m={}
                for l in range(1,k+1):
                    [a_i, a_j]=[i,j]+offset[2*l-2]
                    [b_i, b_j]=[i,j]+offset[2*l-1]
                    context=context+[z_two[a_i,a_j],z_two[b_i,b_j]]
                context_str = ''.join(str(e) for e in context)
                if not m.has_key(context_str):
                    m[context_str]=np.zeros(2,dtype=np.int)
                    m[context_str][z_two[i,j]]=1
                else:
                    m[context_str][z_two[i,j]]+=1
                ratio = float(m[context_str][1]) / float(np.sum(m[context_str]))
                if ratio < th_0: # 0.18
                    s_hat[i,j]=1
                elif ratio >= th_1: # 0.82
                    s_hat[i,j]=2
                else:
                    s_hat[i,j]=0 
                C[(i-1)*(width-2)+j-1]=context_str
        #s_hat=np.reshape(s_hat,(n,))        
        return s_hat, C
    
    elif 6<=k<=12:
        C=zeros((n-2*(width)-2*(width-2)-2*(width-2)-2*(width-4),1),dtype=np.int)
        for i in range(2,width-2):
            for j in range(2,width-2):
                context=[]
                m={}
                for l in range(1,k+1):
                    [a_i, a_j]=[i,j]+offset[2*l-2]
                    [b_i, b_j]=[i,j]+offset[2*l-1]
                    context=context+[z_two[a_i,a_j],z_two[b_i,b_j]]
                context_str = ''.join(str(e) for e in context)
                if not m.has_key(context_str):
                    m[context_str]=np.zeros(2,dtype=np.int)
                    m[context_str][z_two[i,j]]=1
                else:
                    m[context_str][z_two[i,j]]+=1
                    
                ratio = float(m[context_str][1]) / float(np.sum(m[context_str]))
                if ratio < th_0:
                    s_hat[i,j]=1
                elif ratio >= th_1:
                    s_hat[i,j]=2
                else:
                    s_hat[i,j]=0 
                C[(i-2)*(width-4)+j-2]=context_str
        #s_hat=np.reshape(s_hat,(n,))        
        return s_hat, C

In [145]:
m={}
a=[]
a=a+[z_two[9][offset[0,0], offset[0,1]], z_two[9][offset[1,0],offset[1,1]]]
a_str = ''.join(str(e) for e in a)

b=[]
b=b+[0,0,0,0]
b_str=''.join(str(e) for e in b)

In [146]:
print m.has_key(a_str)
print m[a_str][z_two[9][offset[0,0], offset[0,1]]]

False


KeyError: '11'

In [147]:
print z_two[9][offset[0,0], offset[0,1]], z_two[9][offset[1,0],offset[1,1]]
print [z_two[9][offset[0,0], offset[0,1]], z_two[9][offset[1,0],offset[1,1]]]
print a
print a_str
print b
print b_str

print c

1 1
[1, 1]
[1, 1]
11
[0, 0, 0, 0]
0000
[1, 1]


In [148]:
print z_two[9]

[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 1 0 1]
 [0 0 0 ..., 0 1 1]]


In [150]:
test_s_hat1, C1=Two_DUDE(z_two[9],1,0.01,n,offset)
test_s_hat4, C4=Two_DUDE(z_two[9],4,0.01,n,offset)
#test_s_hat11, C11=Two_DUDE(z_two[9],11,0.01,n,offset)

In [151]:
print test_s_hat1, test_s_hat1.shape
print test_s_hat4, test_s_hat4.shape
# this implies that the denoising part is o.k.

[[0 2 2 ..., 2 2 0]
 [0 2 2 ..., 2 2 0]
 [0 2 2 ..., 2 2 0]
 ..., 
 [0 1 1 ..., 1 2 0]
 [0 1 1 ..., 2 1 0]
 [0 1 1 ..., 1 2 0]] (512, 512)
[[0 0 0 ..., 0 0 0]
 [0 2 2 ..., 2 2 0]
 [0 2 2 ..., 2 2 0]
 ..., 
 [0 1 1 ..., 1 2 0]
 [0 1 1 ..., 2 1 0]
 [0 0 0 ..., 0 0 0]] (512, 512)


In [125]:
######## 2-D DUDE Denoising ########
def denoise_with_s_Two_DUDE(z_two,s,k):
    width=int(math.sqrt(z_two.shape[0]*z_two.shape[1]))
    x_hat=z_two.copy()
    s=s.reshape((z_two.shape[0], z_two.shape[1]))
    if k==1:
        for i in range(0,width):
            for j in range(1,width-1):
                if s[i,j]==0:
                    x_hat[i,j]=z_two[i,j]
                elif s[i,j]==1:
                    x_hat[i,j]=0
                else:
                    x_hat[i,j]=1
        #x_hat=x_hat.reshape(z_two.shape[0]*z_two.shape[1],)
        return x_hat
    elif 2<=k<=4:
        for i in range(1,width-1):
            for j in range(1,width-1):
                if s[i,j]==0:
                    x_hat[i,j]=z_two[i,j]
                elif s[i,j]==1:
                    x_hat[i,j]=0
                else:
                    x_hat[i,j]=1
        x_hat=x_hat.reshape(z_two.shape[0]*z_two.shape[1],)
        return x_hat
    elif k==5:
        for i in range(1,width-1):
            for j in range(2,width-2):
                if s[i,j]==0:
                    x_hat[i,j]=z_two[i,j]
                elif s[i,j]==1:
                    x_hat[i,j]=0
                else:
                    x_hat[i,j]=1
        x_hat=x_hat.reshape(z_two.shape[0]*z_two.shape[1],)
        return x_hat
    elif 6<=k<=12:
        for i in range(2,width-2):
            for j in range(2,width-2):
                if s[i,j]==0:
                    x_hat[i,j]=z_two[i,j]
                elif s[i,j]==1:
                    x_hat[i,j]=0
                else:
                    x_hat[i,j]=1
        #x_hat=x_hat.reshape(z_two.shape[0]*z_two.shape[1],)
        return x_hat

In [126]:
x_hat11=denoise_with_s_Two_DUDE(z_two[9],test_s_hat2,11)
x_hat1=denoise_with_s_Two_DUDE(z_two[9],test_s_hat1,1)

In [129]:
print x_hat1, x_hat1.shape
print x_hat11, x_hat11.shape

[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 1 0 1]
 [0 0 0 ..., 0 1 1]] (512, 512)
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 1 0 1]
 [0 0 0 ..., 0 1 1]] (512, 512)


In [24]:
######## 2-D N-DUDE ########
def make_data_for_Two_NN_DUDE(P,Z,k,L,nb_classes,n,offset):
    width=int(math.sqrt(n))
    if k==1:
        # context generation part #
        C=zeros((n-2*width, 2*k*nb_classes))
        for i in range(0,width): # i-th row
            for j in range(1,width-1): # j-th col
                c_i=[]
                for l in range(1,k+1):
                    [a_i, a_j]=[i,j]+offset[2*l-2]
                    [b_i, b_j]=[i,j]+offset[2*l-1]
                    
                    c_i=c_i+P[a_i,a_j,].tolist()+P[b_i,b_j,].tolist()
                C[i*(width-2)+j-1,]=c_i # each row has width-2 cols
        # pseudo label generation part #
        col_cut = np.split(P, [1, width-1], axis=1)
        listing = np.reshape(col_cut[1], (col_cut[1].shape[0]*col_cut[1].shape[1], col_cut[1].shape[2]))
        Y=dot(listing,L)        
        return C,Y
    elif 2<=k<=4:
        C=zeros((n-2*width-2*(width-2), 2*k*nb_classes))
        for i in range(1,width-1): 
            for j in range(1,width-1):
                c_i=[]
                for l in range(1,k+1):
                    [a_i, a_j]=[i,j]+offset[2*l-2]
                    [b_i, b_j]=[i,j]+offset[2*l-1]
                    
                    c_i=c_i+P[a_i,a_j,].tolist()+P[b_i,b_j,].tolist()
                C[(i-1)*(width-2)+j-1,]=c_i
        
        col_cut = np.split(P, [1, width-1], axis=1)
        row_cut = np.split(col_cut[1], [1, width-1])
        listing = np.reshape(row_cut[1], (row_cut[1].shape[0]*row_cut[1].shape[1], row_cut[1].shape[2]))
        Y=dot(listing,L)
        return C,Y

In [31]:
C_NN,Y_NN=make_data_for_Two_NN_DUDE(P,Z,1,L_new[18:20,],nb_classes,n,offset)

In [33]:
print C_NN, C_NN.shape
print Y_NN, Y_NN.shape

[[ 0.  1.  0.  1.]
 [ 0.  1.  0.  1.]
 [ 0.  1.  0.  1.]
 ..., 
 [ 1.  0.  1.  0.]
 [ 0.  1.  0.  1.]
 [ 0.  1.  1.  0.]] (261120, 4)
[[ 1.025  0.     1.25 ]
 [ 1.025  0.     1.25 ]
 [ 1.025  0.     1.25 ]
 ..., 
 [ 1.025  0.     1.25 ]
 [ 1.025  1.25   0.   ]
 [ 1.025  0.     1.25 ]] (261120, 3)
